# Google Spreadsheet Mail Merge
Original template from [`https://github.com/jlumbroso/google-spreadsheet-mail-merge`](https://github.com/jlumbroso/google-spreadsheet-mail-merge).

## 1. 📝 TODO: Customize this part of the notebook
These constants define:
- the spreadsheet ID (long alphanumerical identifier) and worksheet index (0-based);
- the `From:` name and email you would like to use as a sender;
- the default subject line for the emails, if this isn't encoded as a `<title></title>` tag in the templates;
- the folder in which templates are located (shouldn't change!).

In [1]:
# spreadsheet and worksheet to use
SSHEET_KEY = "1ASvG2maFz0Jw9qk44PktzTWjS6Grxdvj_S4NiyIxCTE"
SSHEET_WORKSHEET_ID = 0

# email information
EMAIL_FROM_SENDER = "Domenica Sviola <desviola@princeton.edu>"
EMAIL_DEFAULT_SUBJECT = "Please {first_name}: Could you fill this form out by Wed?"

# email template folder
EMAIL_TEMPLATE_FOLDER = "./templates"

## 2. 🤖 Boilerplate code and helper methods
This section contains the code that is used to connect to the Google Spreadsheet API and send emails through SendGrid. There shouldn't be anything to change here.

Go directly to the third section.

In [2]:
import json
import os
import re
import typing

# THIRD PARTY PACKAGES
import dotenv    # To load environment variables from .env file (https://github.com/theskumar/python-dotenv)
import gspread   # To access Google Sheets (https://gspread.readthedocs.io/en/latest/)
import html2text # To convert HTML to text (https://github.com/Alir3z4/html2text/)
import jinja2    # To render HTML templates (https://jinja.palletsprojects.com/en/2.11.x/)
import sendgrid  # To send emails (https://sendgrid.com/docs/for-developers/sending-email/v3-python-code-example/)
import tqdm      # To display progress bars (https://github.com/tqdm/tqdm)

In [3]:
# load any .env files around
dotenv.load_dotenv()

# api key (must be configured in Codespaces encrypted secrets)
SENDGRID_API_KEY = os.environ.get("SENDGRID_API_KEY")

In [4]:
# sanity check
if SENDGRID_API_KEY is None:
    raise ValueError(
        "SENDGRID_API_KEY is not properly configured; "
        "if you are running this in a Codespace, please configure it in the encrypted secrets."
    )

In [5]:
# creating client using our API key (requires either local file,
# or GitHub secret configuration)

gc = gspread.service_account(filename="service_account.json")

# retrieving spreadsheet

try:
    
    ssheet = gc.open_by_key(SSHEET_KEY)

except gspread.exceptions.APIError as exc:
    if exc.response.status_code == 404:
        raise Exception("Spreadsheet not found") from exc

    elif exc.response.status_code == 403:
        raise Exception(
            "Spreadsheet not accessible: "
            "Either make the spreadsheet publically readable, or add "
            "the email from the service account as a user with read-access."
        ) from exc
    else:
        raise

if ssheet is None:
    raise Exception("Spreadsheet not found")

# printing spreadsheet info

print("Sheet title:", ssheet.title)
print("Worksheets:")
for i, wks in enumerate(ssheet.worksheets()):
    print("->", i, wks.title, "   <== selected" if i == SSHEET_WORKSHEET_ID else "")

Sheet title: Google Spreadsheet Mail Merge — example
Worksheets:
-> 0 emails    <== selected


In [6]:
# retrieving specific worksheet
ws = ssheet.get_worksheet(SSHEET_WORKSHEET_ID)

# get all rows
rows = ws.get_all_values()

# separate header
header = rows[0]
rows = rows[1:]

# zip records
records = [dict(zip(header, row)) for row in rows]

# print sample record
print("Sample record:")
if len(records) > 0:
    print(json.dumps(records[0], indent=2))
else:
    print("No records found")

Sample record:
{
  "name": "Salima El-Amin",
  "university": "Harvard University",
  "email": "google-spreadsheet-mail-merge-test@mailsac.com",
  "group": "4"
}


In [7]:
# See here for a primer on Jinja2:
# https://realpython.com/primer-on-jinja-templating/#render-your-first-jinja-template

environment = jinja2.Environment(loader=jinja2.FileSystemLoader(EMAIL_TEMPLATE_FOLDER))

def render_template(
    template_file_name,
    **variables
):
    template = environment.get_template(template_file_name)
    content = template.render(
        **variables
    )
    return content

In [8]:
def extract_title_from_html(html: str) -> typing.Optional[str]:
    """
    Extract title from an HTML document.

    :param html: The HTML document
    :type html: str

    :return: A title, if one is found; `None` otherwise
    :rtype: typing.Optional[str]
    """
    match = re.search(r"<title>([^<]*)</title>", html)

    # no match found: return empty string
    if match is None:
        return
    
    # match found: extract first group
    raw_title = match.group(1)

    # clean-up title
    title = re.sub(r"\s+", " ", raw_title).strip()

    # If empty string, just return None
    if title == "":
        return

    return title

In [9]:
def send_customized_email(
    email: str,
    template_file_name: str,
    name: typing.Optional[str]=None,
    subject: typing.Optional[str]=None,
    variables: typing.Optional[dict]=None,
    **kwargs
) -> bool:
    """
    Send a customized email.

    :param email: Recipient email address
    :type email: str
    :param template_file_name: Jinja2 template filename to use to render the email body
    :type template_file_name: str
    :param name: Recipient full name
    :type name: typing.Optional[str], optional
    :param subject: Email subject, if not present as <title></title> in the HTML template
    :type subject: typing.Optional[str], optional
    :param variables: Additional variables to pass to the template
    :type variables: typing.Optional[dict], optional
    :raises ValueError: If no email address is provided
    :return: `True` if the email was sent successfully, `False` otherwise
    :rtype: bool
    """

    # exit if no email
    if email is None or email == "":
        raise ValueError("No `email` address provided")

    # determine first name
    first_name = ""
    if name is not None and name != "":
        first_name = name.split()[0]
    
    # to object
    to_obj = email
    if name != "":
        to_obj = (email, name)
    
    # merge all arguments
    template_arguments = {
        "name": name,
        "first_name": first_name,
    }
    template_arguments.update(kwargs)
    if variables is not None:
        template_arguments.update(variables)

    # render template and compute text version
    html_body = render_template(
        template_file_name=template_file_name,
        **template_arguments
    )
    txt_body = html2text.HTML2Text().handle(html_body)

    # try to extract subject from HTML
    extracted_subject = extract_title_from_html(html_body)

    # render subject
    template_arguments = template_arguments or dict()
    email_subject = (
        # provided as argument to function
        (subject.format(**template_arguments) if subject is not None else None)
        # extracted from HTML (template already rendered, so no format())
        or extracted_subject
        # default subject
        or EMAIL_DEFAULT_SUBJECT.format(**template_arguments)
    )

    # create SendGrid message
    message = sendgrid.helpers.mail.Mail(
        from_email=EMAIL_FROM_SENDER,
        to_emails=[to_obj],
        subject=email_subject,
        html_content=html_body,
        plain_text_content=txt_body,
    )

    #message.cc = "Some Person <some@email.com>"
    #message.reply_to = "Some Person <some@email.com>"

    # remove tracking data to make email seem less spammy
    tracking_settings = sendgrid.helpers.mail.TrackingSettings()
    tracking_settings.click_tracking = sendgrid.helpers.mail.ClickTracking(
        enable=False,
        enable_text=False
    )
    tracking_settings.open_tracking = sendgrid.helpers.mail.OpenTracking(
        enable=False
    )
    message.tracking_settings = tracking_settings

    # Add unsubsubscribe header to seem less spammy
    message.add_header(
        sendgrid.helpers.mail.Header("List-Unsubscribe", EMAIL_FROM_SENDER)
    )
    
    # Create the SendGrid client and send the email
    sg = sendgrid.SendGridAPIClient(SENDGRID_API_KEY)
    response = sg.send(message)

    # Print log
    #print(message)
    #print(response.status_code, response.body, response.headers)

    return response.status_code == 202

## 3. 🪄 TODO: Actual logic of how to customize emails

The function `send_customized_email_to_record()` should take a single record — a row of the spreadsheet — and make an appropriate call to `send_customized_email()`. In this example, we are using [our public example spreadsheet](https://docs.google.com/spreadsheets/d/1ASvG2maFz0Jw9qk44PktzTWjS6Grxdvj_S4NiyIxCTE/edit#gid=0), which contains four columns: `name`, `university`, `email`, and `group`.

The `group` column is used to select the template, of the two example templates, as an example of conditional logic.

In [10]:
# customizing the above function to process a row from the Google Spreadsheet
test_record = {
   "name": "Salima El-Amin",
   "university": "Harvard University",
   "email": "google-spreadsheet-mail-merge-test@mailsac.com",
   "group": "4"
}

def send_customized_email_to_record(record):
   template_file_name = "template_example1_commented.html"
   if record["group"] in ["1", "4"]:
       template_file_name = "template_example2_instantiated.html"

   return send_customized_email(
      email=record["email"],
      template_file_name=template_file_name,
      name=record["name"],
      variables=record,
   )

## 4. 🚀 Send the emails
Once the implementation of `send_customized_email_to_record()` is finished, it can be called on all the rows of the spreadsheet.

In [11]:
# send to test record (or send a single email)
send_customized_email_to_record(test_record)

True

In [12]:
# send to all (using TQDM as progress bar)
failed_records = []
for record in tqdm.tqdm(records):
    if not send_customized_email_to_record(record):
        failed_records.append(record)

print("{} total records; {} failed to send".format(
    len(records),
    len(failed_records)
))

100%|██████████| 20/20 [00:01<00:00, 12.31it/s]

20 total records; 0 failed to send
